In [1]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/swarm.git to c:\users\rpdlpascual\appdata\local\temp\pip-req-build-a4ollwrf
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git 'C:\Users\rpdlpascual\AppData\Local\Temp\pip-req-build-a4ollwrf'

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rpdlpascual\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rpdlpascual\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import os
import pandas as pd
import numpy as np
from swarm import Swarm, Agent
from openai import OpenAI
from datetime import datetime

In [19]:
api = os.environ["OPENAI_API_KEY"] = ""

In [20]:
openai_client = OpenAI(api_key=api)

In [21]:
client = Swarm(openai_client)


In [22]:
# Load and prepare the dataset
df = pd.read_csv('dataset.csv')

In [23]:
# Data preprocessing
def preprocess_data(df):
    # Convert revenue strings to numeric
    df['revenue'] = df['revenue'].str.replace('₱', '').str.replace(',', '').astype(float)
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Calculate derived metrics
    df['conversion_rate'] = df['transactions'] / df['visits']
    df['average_order_value'] = df['revenue'] / df['transactions']
    
    return df

df = preprocess_data(df)

In [33]:
def handoff_to_pricing():
    return pricing_analyst

def handoff_to_traffic():
    return traffic_analyst

def handoff_to_sales():
    return sales_analyst


In [40]:
pricing_analyst = Agent(
    name="Pricing Intelligence Analyst",
    system_prompt="""You are a Pricing Intelligence Analyst. Your responsibilities include:
    1. Analyze pricing patterns and revenue metrics
    2. Calculate and interpret average order values
    3. Identify price elasticity patterns
    4. Recommend optimal price points by traffic source
    5. Detect seasonal pricing opportunities
    
    Use statistical analysis to support your recommendations.""",
    model="gpt-4-turbo-preview",  # Updated model name
    tools=[handoff_to_traffic, handoff_to_sales],  # Changed 'functions' to 'tools'
)

traffic_analyst = Agent(
    name="Traffic Analysis Specialist",
    system_prompt="""You are a Traffic Analysis Specialist. Your responsibilities include:
    1. Analyze traffic source performance metrics
    2. Compare conversion rates across channels
    3. Evaluate campaign ROI by source
    4. Identify high-performing traffic segments
    5. Recommend traffic acquisition strategies
    
    Focus on data-driven channel optimization.""",
    model="gpt-4-turbo-preview",
    tools=[handoff_to_pricing, handoff_to_sales],
)

sales_analyst = Agent(
    name="Sales Performance Analyst",
    system_prompt="""You are a Sales Performance Analyst. Your responsibilities include:
    1. Analyze sales patterns and trends
    2. Identify seasonal sales opportunities
    3. Evaluate conversion funnel metrics
    4. Segment customer purchase behavior
    5. Recommend sales optimization strategies
    
    Provide actionable insights backed by data.""",
    model="gpt-4-turbo-preview",
    tools=[handoff_to_pricing, handoff_to_traffic],
)

In [28]:
# Modified data summary with proper scalar values
data_summary = """
Dataset Overview:
- Total Records: {:,}
- Date Range: {} to {}
- Total Revenue: ₱{:,.2f}
- Total Transactions: {:,}
- Average Order Value: ₱{:,.2f}
- Overall Conversion Rate: {:.2%}

Key Performance Indicators:
- Daily Average Revenue: ₱{:,.2f}
- Daily Average Transactions: {:.0f}
- Daily Average Visits: {:.0f}

Analysis Objectives:
1. Revenue Optimization: Identify pricing strategies to maximize revenue
2. Traffic Quality: Evaluate and optimize traffic performance
3. Sales Enhancement: Recommend strategies to improve conversion rates
""".format(
    len(df),
    df['date'].min().strftime('%Y-%m-%d'),
    df['date'].max().strftime('%Y-%m-%d'),
    df['revenue'].sum(),
    df['transactions'].sum(),
    df['revenue'].sum() / df['transactions'].sum(),
    df['transactions'].sum() / df['visits'].sum(),
    df.groupby('date')['revenue'].mean().mean(),  # Convert Series to scalar
    df.groupby('date')['transactions'].mean().mean(),  # Convert Series to scalar
    df.groupby('date')['visits'].mean().mean()  # Convert Series to scalar
)

In [29]:
# Initialize conversation structure
struct = [{"role": "user", "content": data_summary}]

In [41]:
print("Starting multi-agent analysis...")
for turn in range(6):
    current_agent = {
        0: pricing_analyst,
        1: traffic_analyst,
        2: sales_analyst
    }[turn % 3]
    
    print(f"\nTurn {turn + 1}: {current_agent.name} analyzing...")
    response = client.run(agent=current_agent, messages=struct)
    
    # Add the response directly to the conversation structure
    struct.append({
        "role": "assistant",
        "content": f"{current_agent.name} (Turn {turn + 1}):\n{response}"
    })

Starting multi-agent analysis...

Turn 1: Pricing Intelligence Analyst analyzing...

Turn 2: Traffic Analysis Specialist analyzing...

Turn 3: Sales Performance Analyst analyzing...

Turn 4: Pricing Intelligence Analyst analyzing...

Turn 5: Traffic Analysis Specialist analyzing...

Turn 6: Sales Performance Analyst analyzing...


In [42]:
# Enhanced results presentation
print("\n=== E-commerce Analysis Results ===")
for i, message in enumerate(struct):
    if i == 0:
        print("\nInitial Data Summary:")
    else:
        print(f"\nInsight {i} - {message['role'].title()}:")
    print(message['content'])
    print("\n" + "="*80)


=== E-commerce Analysis Results ===

Initial Data Summary:

Dataset Overview:
- Total Records: 52,721
- Date Range: 2020-01-01 to 2020-09-30
- Total Revenue: ₱6,724,818,522.00
- Total Transactions: 1,058,977
- Average Order Value: ₱6,350.30
- Overall Conversion Rate: 15.80%

Key Performance Indicators:
- Daily Average Revenue: ₱126,839.23
- Daily Average Transactions: 20
- Daily Average Visits: 126

Analysis Objectives:
1. Revenue Optimization: Identify pricing strategies to maximize revenue
2. Traffic Quality: Evaluate and optimize traffic performance
3. Sales Enhancement: Recommend strategies to improve conversion rates



Insight 1 - Assistant:
Pricing Intelligence Analyst (Turn 1):
Based on the dataset overview and key performance indicators provided, here are the analyses and recommendations for revenue optimization, traffic quality evaluation, and sales enhancement:

### 1. Revenue Optimization

To identify pricing strategies that maximize revenue, it's essential to analyze pric